[som](https://github.com/darecophoenixx/wordroid.sblo.jp/wiki/%5Bsom%5D-scikit-learn-api)

* Kaggle notebook は、それほど良い性能ではないようだ。
* ver6からsksom_keras2へ変更（GPU=off）

```
CPU times: user 51min 36s, sys: 25min 54s, total: 1h 17min 30s
Wall time: 21min 8s
```

* 遅い場合はGPUをonにしてみてください
* ver7からGPU=on

```
CPU times: user 42.4 s, sys: 8.2 s, total: 50.6 s
Wall time: 51.2 s
```

In [ ]:
!pip install japanize-matplotlib

In [ ]:
!pip install git+https://github.com/darecophoenixx/wordroid.sblo.jp

In [ ]:
%matplotlib inline
from IPython.display import SVG, Image
from keras.utils.vis_utils import model_to_dot

import random

import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn import preprocessing
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.style.use('seaborn')
%config InlineBackend.figure_format = 'retina'

In [ ]:
import japanize_matplotlib
plt.rcParams['font.family']

In [ ]:
img_cnt = 0

In [ ]:
def save_img():
    global img_cnt
    img_cnt += 1
    img_name = 'img_{0:03}.png'.format(img_cnt)
    print(img_name)
    plt.savefig(img_name)

In [ ]:
def hexbin(x, y, color, **kwargs):
    cmap = sns.light_palette(color, as_cmap=True)
    plt.hexbin(x, y, cmap=cmap, **kwargs)
def scatter(x, y, color, **kwargs):
    plt.scatter(x, y, marker='.')

In [ ]:
df_part = pd.read_csv('../input/sample-data-wordanddoc2vec/actual_survey_data_jp.csv', index_col=0)
df_part.head()

In [ ]:
df_out_wgt_user = pd.read_csv('../input/lowcol-actual-survey-data-fet12-randinit01/out_user.csv', index_col=0)
X_df_user = df_out_wgt_user
wgt_user = df_out_wgt_user.values
df_out_wgt_user.head()

In [ ]:
df_out_wgt_col = pd.read_csv('../input/lowcol-actual-survey-data-fet12-randinit01/out_col.csv', index_col=0)
X_df_col = df_out_wgt_col
wgt_prod = df_out_wgt_col.values
df_out_wgt_col.head()

In [ ]:
som_out_user = pd.read_csv('../input/lowcol-actual-survey-data-fet12-randinit01-somcls/som_out_user.csv', index_col=0)
som_out_col = pd.read_csv('../input/lowcol-actual-survey-data-fet12-randinit01-somcls/som_out_col.csv', index_col=0)
som_out_user.head()

In [ ]:
kshape1 = 4

X_df = df_out_wgt_user
X_df['som0'] = np.array([divmod(c, kshape1) for c in som_out_user['cls'].values])[:,0]
X_df['som1'] = np.array([divmod(c, kshape1) for c in som_out_user['cls'].values])[:,1]
X_df_col['som0'] = np.array([divmod(c, kshape1) for c in som_out_col['cls'].values])[:,0]
X_df_col['som1'] = np.array([divmod(c, kshape1) for c in som_out_col['cls'].values])[:,1]
X_df.head()

In [ ]:
X_df_col.head()

In [ ]:
from som import som
from som.keras import sksom_keras, sksom_keras2

K_shape = (30, 40)
sinit = som.SimpleSOM(K_shape, initialization_func='linear')
sinit._initialize(X_df.values)

In [ ]:
img = som.conv2img(sinit.K, K_shape)
plt.figure(figsize=(10, 10))
plt.imshow(img)
save_img()

In [ ]:
K_init = sinit.K
sobj = sksom_keras2(K_shape, init_K=K_init)

In [ ]:
df1= pd.DataFrame(sobj.landmarks_[:,:5])
df1['cls'] = 'K'
df1.head()
df2 = pd.DataFrame(X_df.values[:,:5])
df2['cls'] = 'X'
df2.head()
df = pd.concat([df2, df1], axis=0)
df.head()
df.shape
sns.pairplot(df, markers=['.', 's'], hue='cls', plot_kws={'alpha': 0.5}, diag_kind='hist')
save_img()

In [ ]:
som_xy = sobj.label2xy(sobj.predict(X_df.values))
som_xy

plt.figure(figsize=(12,12))
ax = sns.kdeplot(som_xy[:,0], som_xy[:,1],
                 cmap="Blues", shade=True, thresh=0.05)

for ii, c in enumerate(sobj.predict(X_df_col.values)):
    colname = X_df_col.index.values[ii]
    text = colname
    xy = sobj.label2xy([c])[0,:]
    ax.scatter(xy[0], xy[1], c='darkgreen')
    ax.annotate(text, xy=xy)
save_img()

In [ ]:
%%time
print(sobj.r)
hst = sobj.fit(X_df.values, batch_size=X_df.values.shape[0])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,8))
ax.plot(list(range(len(sobj.hst['MeanDist2ClosestLM']))), sobj.hst['MeanDist2ClosestLM'], label="MeanDist2ClosestLM")
plt.legend(loc="best")
save_img()

In [ ]:
img = som.conv2img(sobj.landmarks_, K_shape)
plt.figure(figsize=(10, 10))
plt.imshow(img)
save_img()

In [ ]:
img = som.conv2img(sobj.landmarks_, K_shape, target=[12,13,0])
plt.figure(figsize=(10, 10))
plt.imshow(img)
save_img()

In [ ]:
img = som.conv2img(sobj.landmarks_, K_shape)
plt.figure(figsize=(20, 20))
plt.imshow(img)

for i, m in enumerate(sobj.predict(X_df_col.values)):
    b, a = divmod(m, sobj.kshape[1])
    txt = X_df_col.index.values[i]
    plt.text(a, b, txt, ha='center', va='center',
           bbox=dict(facecolor='lightblue', alpha=0.5, lw=0))
save_img()

In [ ]:
sobj.predict(X_df.values)

In [ ]:
df1= pd.DataFrame(sobj.landmarks_[:,:5])
df1['cls'] = 'K'
df1.head()
df2 = pd.DataFrame(X_df.values[:,:5])
df2['cls'] = 'X'
df2.head()
df = pd.concat([df2, df1], axis=0)
df.head()
df.shape
sns.pairplot(df, markers=['.', 's'], hue='cls', plot_kws={'alpha': 0.5}, diag_kind='hist')
save_img()

In [ ]:
%%time
from keras.optimizers import RMSprop, Adam, SGD

'''002'''
hst = sobj.fit(X_df.values, r=(K_shape[0]/30.0, K_shape[0]/30.0), optimizer=Adam(learning_rate=0.001), batch_size=X_df.values.shape[0],
               epochs=1500)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,8))
ax.plot(list(range(len(sobj.hst['MeanDist2ClosestLM']))), sobj.hst['MeanDist2ClosestLM'], label="MeanDist2ClosestLM")
plt.legend(loc="best")
save_img()

In [ ]:
img = som.conv2img(sobj.landmarks_, K_shape)
plt.figure(figsize=(10, 10))
plt.imshow(img)
save_img()

In [ ]:
img = som.conv2img(sobj.landmarks_, K_shape, target=[12,13,0])
plt.figure(figsize=(10, 10))
plt.imshow(img)
save_img()

In [ ]:
img = som.conv2img(sobj.landmarks_, K_shape)
plt.figure(figsize=(20, 20))
plt.imshow(img)

for i, m in enumerate(sobj.predict(X_df_col.values)):
    b, a = divmod(m, sobj.kshape[1])
    txt = X_df_col.index.values[i]
    plt.text(a, b, txt, ha='center', va='center',
           bbox=dict(facecolor='lightblue', alpha=0.5, lw=0))
save_img()

In [ ]:
df1= pd.DataFrame(sobj.landmarks_[:,:5])
df1['cls'] = 'K'
df1.head()
df2 = pd.DataFrame(X_df.values[:,:5])
df2['cls'] = 'X'
df2.head()
df = pd.concat([df2, df1], axis=0)
df.head()
df.shape
sns.pairplot(df, markers=['.', 's'], hue='cls', plot_kws={'alpha': 0.5}, diag_kind='hist')
save_img()

In [ ]:
som_xy = []
for ii, c in enumerate(sobj.predict(X_df.values)):
    b, a = divmod(c, sobj.kshape[1])
    som_xy.append((b, a))
som_xy = np.array(som_xy)
som_xy

plt.figure(figsize=(15,15))
ax = sns.kdeplot(som_xy[:,1], som_xy[:,0],
                 cmap="Blues", shade=True, thresh=0.05)

for ii, c in enumerate(sobj.predict(X_df_col.values)):
    colname = X_df_col.index.values[ii]
    text = colname
    b, a = divmod(c, sobj.kshape[1])
    xy = (a, b)
    ax.scatter(a, b, c='darkgreen')
    ax.annotate(text, xy=xy)
save_img()

In [ ]:
plt.figure(figsize=(13,9))
ax = sns.kdeplot(som_xy[:,1], som_xy[:,0],
                 cmap="Blues", shade=True, thresh=0.05)

for ii, c in enumerate(sobj.predict(X_df_col.values)):
    colname = X_df_col.index.values[ii]
    text = colname
    b, a = divmod(c, sobj.kshape[1])
    xy = (a, b)
    ax.scatter(a, b, c='darkgreen', s=200, marker='x')
    ax.annotate(text, xy=xy, size=15)
save_img()

In [ ]:
df_out_wgt_user['sommap_cls'] = sobj.predict(X_df.values)
df_out_wgt_col['sommap_cls'] = sobj.predict(X_df_col.values)

df_out_wgt_user.to_csv('sommap_out_user.csv')
df_out_wgt_col.to_csv('sommap_out_col.csv')
pd.DataFrame(sobj.landmarks_).to_csv('sommap_out_LM.csv')